test cells continuously?


In [ ]:
var importer = require('../Core');
var path = require('path');
var Module = require('module').Module;
var Mocha = require('mocha');

// call test runner
var mocha;
if(typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'tdd',
        reporter: 'list'
    });
}

function resetTests(suite) {
    suite.tests.forEach(function(t) {
        delete t.state;
        t.timedOut = false;
    });
    suite.suites.forEach(resetTests);
    suite.tests = [];
}

var testCells = (cells, ctx = {}) => {
    cells.forEach(r => {
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        Object.assign(ctx, Mocha);
        var required = r.runInNewContext(ctx);
        Object.assign(ctx, required);
        mocha.suite.emit('require', required, r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);    
    });
    return new Promise(resolve => mocha.run(function (failures) {
        resetTests(mocha.suite);
        resolve(failures);
    }));
}
module.exports = testCells;



In [4]:
var importer = require('../Core');
$$.async();
// put them in the order to include them in the context
var cells;
importer.interpret(['analyze zuora', 'test zuora'])
    .then(r => cells = r)
    .then(() => importer.import('test cells continuously'))
    .then(testCells => testCells(cells))
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))



  1) zuora to eloqua "before each" hook for "should transfer data end-to-end"
  √ zuora oauth should connect to zuora using oauth: 0ms
  √ zuora export should export a CSV from zuora: 0ms
  √ zuora export should convert CSV to JSON: 0ms
  √ map zuora data fields should map basic data: 0ms
  √ map zuora data fields should map contact data: 0ms
  √ map zuora data fields should map support data: 0ms
  √ map zuora data fields should map cancelled data: 0ms
  √ eloqua bulk upload should authenticate using eloqua oauth: 4ms
  √ eloqua bulk upload should update data to eloqua: 0ms
  √ eloqua bulk upload should confirm upload was successful: 0ms

  10 passing (11ms)
  1 failing

  1) zuora to eloqua
       "before each" hook for "should transfer data end-to-end":
     Error: Cannot find module 'xlsx'
      at require (internal/module.js:11:18)
      at Context.beforeEach (C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\zuora to eloqua.ipynbzuora to eloqua.ipynb[1]:8:20)





1

In [2]:
var importer = require('../Core');
var framework = require('wdio-mocha-framework');
var wdio = require('wdio-sync');
var Module = require('module').Module;
var Mocha = require('mocha');

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE;
var INTERFACES = {
    bdd: ['before', 'beforeEach', 'it', 'after', 'afterEach'],
    tdd: ['suiteSetup', 'setup', 'test', 'suiteTeardown', 'teardown'],
    qunit: ['before', 'beforeEach', 'test', 'after', 'afterEach']
}

var mocha;
if(typeof mocha === 'undefined') {
    mocha = new Mocha({
        ui: 'bdd',
        compilers: ['js:babel-register'],
        timeout: 100000,
        reporter: 'list'
    });
}

$$.async();
// put them in the order to include them in the context
var f = PROFILE_PATH + '/Selenium/ecommerce/test/specs/trial.js';
var ctx = {};
importer.import([
    'test cells continuously',
    'webdriver client'
], {mocha})
    .then(funcs => {
    console.log(funcs)
    //    const adapter = new framework.MochaAdapter(cid, config, specs, capabilities)
    //    return adapter.run()

        ctx.browser = ctx.client = funcs.createWebdriverClient();
        Object.assign(ctx.client.options, {mochaOpts: mocha.options})
    
        const r = {
            id: f,
            runInNewContext: (ctx) => {
                return importer.runInNewContext(fs.readFileSync(f).toString(), Object.assign(ctx, {
                    __filename: f
                }))
            }
        }
        
        mocha.suite.emit('pre-require', ctx, r.id, mocha);
        Object.assign(ctx, Mocha);
        mocha.suite.emit('require',
                         Object.assign(ctx, r.runInNewContext(ctx)),
                         r.id, mocha);
        mocha.suite.emit('post-require', ctx, r.id, mocha);  
    
        wdio.wrapCommands(
            ctx.client,
            ctx.client.options.beforeCommand,
            ctx.client.options.afterCommand)
    
        mocha.suite.on('pre-require', (context, file, m) => {
            //mocha.options(mocha.options, {
            //    context, file, mocha: m, options: mocha.options
            //})

            INTERFACES[mocha.options.ui].forEach((fnName) => {
                let testCommand = INTERFACES[mocha.options.ui][2]

                wdio.runInFiberContext(
                    [testCommand, testCommand + '.only'],
                    ctx.client.options.beforeHook,
                    ctx.client.options.afterHook,
                    fnName,
                    ctx
                )
            })
        })
        return new Promise(resolve => mocha.run(function (failures) {
            resetTests(mocha.suite);
            resolve(failures);
        }));
    })
    .then(r => $$.sendResult(r))
    .catch(e => $$.sendError(e))


hit
[ { code: 'var importer = require(\'../Core\');\nvar path = require(\'path\');\nvar Module = require(\'module\').Module;\nvar Mocha = require(\'mocha\');\n\n// call test runner\nvar mocha;\nif(typeof mocha === \'undefined\') {\n    mocha = new Mocha({\n        ui: \'tdd\',\n        reporter: \'list\'\n    });\n}\n\nfunction resetTests(suite) {\n    suite.tests.forEach(function(t) {\n        delete t.state;\n        t.timedOut = false;\n    });\n    suite.suites.forEach(resetTests);\n    suite.tests = [];\n}\n\nvar testCells = (cells, ctx = {}) => {\n    cells.forEach(r => {\n        mocha.suite.emit(\'pre-require\', ctx, r.id, mocha);\n        Object.assign(ctx, Mocha);\n        var required = r.runInNewContext(ctx);\n        Object.assign(ctx, required);\n        mocha.suite.emit(\'require\', required, r.id, mocha);\n        mocha.suite.emit(\'post-require\', ctx, r.id, mocha);    \n    });\n    return new Promise(resolve => mocha.run(function (failures) {\n        resetTests(moch

Promise { <pending> }